### Imports

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, color
from skimage.feature import local_binary_pattern
from skimage.io import imread
import os
import pickle
import cv2
import pandas as pd

from src.utils.images import load_images_from_directory, transform_images_color_space, split_image_into_quadrants
from src.utils.distance_matrix import create_distance_matrix, generate_results
from src.utils.ml_metrics import mapk

### Constants

In [19]:
query_folder = "./data/qsd1_w3/non_augmented/"
bbdd_folder = "./data/BBDD"

### Functions

In [20]:
def rescale_images(images, image_size):
    rescaled_images = []
    for image in images:
        rescaled_images.append(cv2.resize(image, image_size))
    return rescaled_images

def compute_block_lbp(block, radius, n_points, method):
    """
    Compute the LBP histogram for a single block of the image.
    """
    lbp = local_binary_pattern(block, n_points, radius, method=method)
    
    if method=="uniform":
        # Compute LBP histogram with uniform method (bins range from 0 to n_points + 2)
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), density=True)
    else:
        # Compute LBP histogram with default or ror method
        hist, _ = np.histogram(lbp.ravel(), bins=2**n_points, density=True)
        
    return hist

def compute_images_block_lbp(images, radius, n_points, method, n_blocks):
    """
    Given a list of images, compute the LBP for each image by dividing
    each image into blocks, computing the LBP for each block, and concatenating
    the histograms of all blocks.
    
    Parameters:
    - images: list of loaded images (grayscale)
    - radius: radius for the LBP
    - n_points: number of neighbors (points) for the LBP
    - method: method for LBP ('uniform', 'ror', etc.)
    - n_blocks: number of blocks to divide the image into (1, 4, 16, 64)
    
    Returns:
    - List of concatenated histograms for each image.
    """
    image_histograms = []
    
    for image in images:
        
        # Divide the image into blocks
        blocks = split_image_into_quadrants(image, int(np.emath.logn(4, n_blocks))+1)
        
        # Compute LBP for each block and concatenate histograms
        concatenated_histogram = []
        for block in blocks:
            block_histogram = compute_block_lbp(block, radius, n_points, method)
            concatenated_histogram.append(block_histogram)
        
        # Concatenate all block histograms into a single feature vector for the image
        concatenated_histogram = np.concatenate(concatenated_histogram)
        image_histograms.append([concatenated_histogram.astype('float32')])
    
    return image_histograms

def compute_images_block_multi_lbp(images, method, n_blocks):
    image_histograms = []
    
    for image in images:
        
        # Divide the image into blocks
        blocks = split_image_into_quadrants(image, int(np.emath.logn(4, n_blocks))+1)
        
        # Compute LBP for each block and concatenate histograms
        concatenated_histogram = []
        for block in blocks:
            
            block_histogram = compute_block_lbp(block, 1, 8, method)
            concatenated_histogram.append(block_histogram)
            
            block_histogram = compute_block_lbp(block, 2, 8, method)
            concatenated_histogram.append(block_histogram)
            
            block_histogram = compute_block_lbp(block, 3, 12, method)
            concatenated_histogram.append(block_histogram)
        
        # Concatenate all block histograms into a single feature vector for the image
        concatenated_histogram = np.concatenate(concatenated_histogram)
        image_histograms.append([concatenated_histogram.astype('float32')])
    
    return image_histograms

In [21]:
def lbp_study(query_folder, bbdd_folder,
              colors = ["gray", "L", "V"],
              rescale_list = [True],
              radius_list = [1, 2, 3],
              points_list = [8, 10, 12],
              methods = ["default", "ror", "uniform"],
              num_blocks_list = [1, 4, 16, 64],
              distance_measures = ['intersection', 'correlation', 'bhattacharyya']):
    
    # Store results in a list that will later be converted to a dataframe
    full_results = []
    
    # Load query images
    query_images = load_images_from_directory(query_folder)

    # Load bbdd images
    bbdd_images = load_images_from_directory(bbdd_folder)
    
    # Load groundtruth
    gt_dir = os.path.join(query_folder, 'gt_corresps.pkl')
    with open(gt_dir, 'rb') as reader:
        ground_truth = pickle.load(reader)
    
    for color in colors:
        
        query_images_color = transform_images_color_space(query_images, color_space=color)
        bbdd_images_color = transform_images_color_space(bbdd_images, color_space=color)
        
        for rescale in rescale_list:
            
            if rescale:
                query_images_color = rescale_images(query_images_color, (256, 256))
                bbdd_images_color = rescale_images(bbdd_images_color, (256, 256))
        
            for radius, points in zip(radius_list, points_list):

                for method in methods:

                    for num_blocks in num_blocks_list:
                        
                        query_lbp_vectors = compute_images_block_lbp(query_images_color, radius, points, method, num_blocks)
                        bbdd_lbp_vectors = compute_images_block_lbp(bbdd_images_color, radius, points, method, num_blocks)
                        
                        for distance_measure in distance_measures:
                            
                            # Print current combination
                            print(f"{color}, rescale={rescale}, {radius}-{points}, {method}, {num_blocks}, {distance_measure}", end=" ")

                            # Calculate distance matrix
                            distance_matrix = create_distance_matrix(query_lbp_vectors, 
                                                                     bbdd_lbp_vectors,
                                                                     distance_measure,
                                                                     normalize=None)
                            # Generate sorted results
                            results = generate_results(distance_matrix, distance_measure)

                            # Calculate metrics
                            mapk_val_1 = mapk(ground_truth, results, 1)
                            mapk_val_5 = mapk(ground_truth, results, 5)

                            # Print metrics for current combination
                            print(f"-- MAPK@{1}: {mapk_val_1} -- MAPK@{5}: {mapk_val_5}")

                            # Save current combination parameters and results
                            combination_results = {}
                            combination_results['Color'] = color
                            combination_results['Rescale'] = rescale
                            combination_results['Radius'] = radius
                            combination_results['Points'] = points
                            combination_results['Method'] = method
                            combination_results['Blocks'] = num_blocks
                            combination_results['Distance'] = distance_measure
                            combination_results['MAP@1'] = mapk_val_1
                            combination_results['MAP@5'] = mapk_val_5
                            full_results.append(combination_results)
                    
    # Save results to csv via df
    full_results_df = pd.DataFrame(full_results)
    full_results_df.to_csv("lbp_study_results.csv", index=False)
    
    return full_results_df

In [46]:
results_df = lbp_study(query_folder, bbdd_folder,
              colors = ["gray", "L", "V"],
              rescale_list = [True],
              radius_list = [1, 2, 3],
              points_list = [8, 10, 12],
              methods = ["default", "ror", "uniform"],
              num_blocks_list = [1, 4, 16, 64],
              distance_measures = ['intersection', 'correlation', 'bhattacharyya'])

gray, rescale=True, 1-8, default, 1, intersection -- MAPK@1: 0.23333333333333334 -- MAPK@5: 0.2861111111111111
gray, rescale=True, 1-8, default, 1, correlation -- MAPK@1: 0.23333333333333334 -- MAPK@5: 0.2816666666666666
gray, rescale=True, 1-8, default, 1, bhattacharyya -- MAPK@1: 0.3 -- MAPK@5: 0.325
gray, rescale=True, 1-8, default, 4, intersection -- MAPK@1: 0.4 -- MAPK@5: 0.45111111111111113
gray, rescale=True, 1-8, default, 4, correlation -- MAPK@1: 0.4 -- MAPK@5: 0.4305555555555555
gray, rescale=True, 1-8, default, 4, bhattacharyya -- MAPK@1: 0.4666666666666667 -- MAPK@5: 0.5055555555555555
gray, rescale=True, 1-8, default, 16, intersection -- MAPK@1: 0.6 -- MAPK@5: 0.6261111111111111
gray, rescale=True, 1-8, default, 16, correlation -- MAPK@1: 0.5666666666666667 -- MAPK@5: 0.5833333333333334
gray, rescale=True, 1-8, default, 16, bhattacharyya -- MAPK@1: 0.6 -- MAPK@5: 0.6261111111111111
gray, rescale=True, 1-8, default, 64, intersection -- MAPK@1: 0.8333333333333334 -- MAPK@5: 

L, rescale=True, 2-10, ror, 16, correlation -- MAPK@1: 0.5 -- MAPK@5: 0.5344444444444444
L, rescale=True, 2-10, ror, 16, bhattacharyya -- MAPK@1: 0.5 -- MAPK@5: 0.5083333333333333
L, rescale=True, 2-10, ror, 64, intersection -- MAPK@1: 0.6333333333333333 -- MAPK@5: 0.7194444444444444
L, rescale=True, 2-10, ror, 64, correlation -- MAPK@1: 0.7 -- MAPK@5: 0.7483333333333333
L, rescale=True, 2-10, ror, 64, bhattacharyya -- MAPK@1: 0.6 -- MAPK@5: 0.6316666666666666
L, rescale=True, 2-10, uniform, 1, intersection -- MAPK@1: 0.1 -- MAPK@5: 0.12777777777777777
L, rescale=True, 2-10, uniform, 1, correlation -- MAPK@1: 0.1 -- MAPK@5: 0.11777777777777777
L, rescale=True, 2-10, uniform, 1, bhattacharyya -- MAPK@1: 0.1 -- MAPK@5: 0.14
L, rescale=True, 2-10, uniform, 4, intersection -- MAPK@1: 0.3 -- MAPK@5: 0.3233333333333333
L, rescale=True, 2-10, uniform, 4, correlation -- MAPK@1: 0.26666666666666666 -- MAPK@5: 0.3
L, rescale=True, 2-10, uniform, 4, bhattacharyya -- MAPK@1: 0.3 -- MAPK@5: 0.34166

In [25]:
def multiple_lbp(num_blocks_list = [16, 64],
                 color = "L",
                 rescale = True,
                 radius = "Multi",
                 points = "Multi",
                 method = "default",
                 distance_measure = "bhattacharyya"):
    
    # Save results in a list, which will then be transformed into a df
    full_results = []
    
    # Load query images
    query_images = load_images_from_directory(query_folder)

    # Load bbdd images
    bbdd_images = load_images_from_directory(bbdd_folder)
    
    # Load groundtruth
    gt_dir = os.path.join(query_folder, 'gt_corresps.pkl')
    with open(gt_dir, 'rb') as reader:
        ground_truth = pickle.load(reader)
        
    query_images_color = transform_images_color_space(query_images, color_space=color)
    bbdd_images_color = transform_images_color_space(bbdd_images, color_space=color)
    
    query_images_color = rescale_images(query_images_color, (256, 256))
    bbdd_images_color = rescale_images(bbdd_images_color, (256, 256))
    
    for num_blocks in num_blocks_list:
        
        query_lbp_vectors = compute_images_block_multi_lbp(query_images_color, method, num_blocks)
        bbdd_lbp_vectors = compute_images_block_multi_lbp(bbdd_images_color, method, num_blocks)

        print(f"{num_blocks}", end=" ")

        # Calculate distance matrix
        distance_matrix = create_distance_matrix(query_lbp_vectors, 
                                                 bbdd_lbp_vectors,
                                                 distance_measure,
                                                 normalize=None)
        # Generate sorted results
        results = generate_results(distance_matrix, distance_measure)

        # Calculate metrics
        mapk_val_1 = mapk(ground_truth, results, 1)
        mapk_val_5 = mapk(ground_truth, results, 5)

        # Print metrics for current combination
        print(f"-- MAPK@{1}: {mapk_val_1} -- MAPK@{5}: {mapk_val_5}")

        # Save current combination parameters and results
        combination_results = {}
        combination_results['Color'] = color
        combination_results['Rescale'] = rescale
        combination_results['Radius'] = radius
        combination_results['Points'] = points
        combination_results['Method'] = method
        combination_results['Blocks'] = num_blocks
        combination_results['Distance'] = distance_measure
        combination_results['MAP@1'] = mapk_val_1
        combination_results['MAP@5'] = mapk_val_5
        full_results.append(combination_results)
    
    # Save results
    full_results_df = pd.DataFrame(full_results)
    full_results_df.to_csv("multiple_lbp_results.csv", index=False)
    
    return full_results_df

In [24]:
multiple_lbp_results_df = multiple_lbp()

16 -- MAPK@1: 0.8333333333333334 -- MAPK@5: 0.9055555555555554
64 -- MAPK@1: 1.0 -- MAPK@5: 1.0


In [26]:
!pip list

Package                            Version



[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


---------------------------------- --------------------
absl-py                            1.0.0
affine                             2.4.0
alabaster                          0.7.12
altgraph                           0.17
anaconda-client                    1.7.2
anaconda-navigator                 2.3.2
anaconda-project                   0.9.1
annoy                              1.17.1
anyio                              2.2.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.5
astropy                            5.2.2
astunparse                         1.6.3
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
auto-py-to-exe                     2.22.0
Automat                            20.2.0
autopep8                           1.5.6
Babel                              2.9.